##### Copyright 2020 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2020 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/boundless"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/hub/tutorials/boundless.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/hub/tutorials/boundless.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/hub/tutorials/boundless.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
  <td><a href="https://tfhub.dev/s?q=google%2Fboundless"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF Hub 모델보기</a></td>
</table>

# Boundless Colab

Boundless 모델 Colab에 오신 것을 환영합니다! 이 노트북은 이미지에서 모델을 실행하고 결과를 시각화하는 단계를 안내합니다.

## 개요

Boundless는 이미지 외삽을 위한 모델입니다. 이 모델은 이미지를 가져와서 일부([1/2](https://tfhub.dev/google/boundless/half/1), [1/4](https://tfhub.dev/google/boundless/quarter/1), [3/4](https://tfhub.dev/google/boundless/three_quarter/1))를 내부적으로 마스킹하고 마스킹된 부분을 완성합니다. 자세한 내용은 [Boundless: Generative Adversarial Networks for Image Extension](https://arxiv.org/pdf/1908.07007.pdf) 또는 TensorFlow Hub의 모델 문서를 참조하세요.

## 가져오기 및 설정

기본 가져오기부터 시작하겠습니다.

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from io import BytesIO
from PIL import Image as PilImage
import numpy as np
from matplotlib import pyplot as plt
from six.moves.urllib.request import urlopen

## 입력을 위한 이미지 읽기

이미지를 로드하고 모델에 맞게 형식(257x257x3)을 지정하는 데 도움을 주는 util 메서드를 만들어보겠습니다. 이 메서드는 또한 왜곡을 피하기 위해 이미지를 정사각형으로 자릅니다. 로컬 이미지 또는 인터넷에서 찾은 이미지를 사용할 수 있습니다.

In [ ]:
  def read_image(filename):
    fd = None
    if(filename.startswith('http')):
      fd = urlopen(filename)
    else:
      fd = tf.io.gfile.GFile(filename, 'rb')

    pil_image = PilImage.open(fd)
    width, height = pil_image.size
    # crop to make the image square
    pil_image = pil_image.crop((0, 0, height, height))
    pil_image = pil_image.resize((257,257),PilImage.ANTIALIAS)
    image_unscaled = np.array(pil_image)
    image_np = np.expand_dims(
        image_unscaled.astype(np.float32) / 255., axis=0)
    return image_np

## 시각화 메서드

모델에 의해 생성된 마스킹된 버전 및 "채워진" 버전과 함께 원본 이미지를 나란히 표시하는 시각화 메서드도 생성합니다.

In [ ]:
def visualize_output_comparison(img_original, img_masked, img_filled):
  plt.figure(figsize=(24,12))
  plt.subplot(131)
  plt.imshow((np.squeeze(img_original)))
  plt.title("Original", fontsize=24)
  plt.axis('off')
  plt.subplot(132)
  plt.imshow((np.squeeze(img_masked)))
  plt.title("Masked", fontsize=24)
  plt.axis('off')
  plt.subplot(133)
  plt.imshow((np.squeeze(img_filled)))
  plt.title("Generated", fontsize=24)
  plt.axis('off')
  plt.show()

## 이미지 로드하기

여기서는 샘플 이미지를 로드하지만 고유한 이미지를 colab에 업로드하고 사용해 볼 수 있습니다. 모델에는 사람 이미지와 관련하여 몇 가지 제한 사항이 있음을 염두에 두세요.

In [ ]:
wikimedia = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Nusfjord_road%2C_2010_09.jpg/800px-Nusfjord_road%2C_2010_09.jpg"
# wikimedia = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/Beech_forest_M%C3%A1tra_in_winter.jpg/640px-Beech_forest_M%C3%A1tra_in_winter.jpg"
# wikimedia = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/Marmolada_Sunset.jpg/640px-Marmolada_Sunset.jpg"
# wikimedia = "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Aegina_sunset.jpg/640px-Aegina_sunset.jpg"

input_img = read_image(wikimedia)

## TensorFlow Hub에서 모델 선택하기

TensorFlow Hub에는 Half, Quarter 및 Three Quarters의 3가지 버전의 Boundless 모델이 있습니다. 다음 셀에서 그 중 하나를 선택하고 자신의 이미지에 시도해볼 수 있습니다. 다른 것을 시도하고 싶다면 간단히 선택하고 다음 셀을 실행하세요.

In [ ]:
#@title Model Selection { display-mode: "form" }
model_name = 'Boundless Quarter' # @param ['Boundless Half', 'Boundless Quarter', 'Boundless Three Quarters']
model_handle_map = {
    'Boundless Half' : 'https://tfhub.dev/google/boundless/half/1',
    'Boundless Quarter' : 'https://tfhub.dev/google/boundless/quarter/1', 
    'Boundless Three Quarters' : 'https://tfhub.dev/google/boundless/three_quarter/1'
}

model_handle = model_handle_map[model_name]


이제 원하는 모델을 선택했으므로 TensorFlow Hub에서 로드하겠습니다.

**참고**: 브라우저에서 모델 핸들을 가리켜 모델 설명서를 읽을 수 있습니다.

In [ ]:
print("Loading model {} ({})".format(model_name, model_handle))
model = hub.load(model_handle)

## 추론하기

Boundless 모델에는 다음 두 가지 출력이 있습니다.

- 마스크가 적용된 입력 이미지
- 이미지 완성을 위해 내삽이 포함된 마스킹된 이미지

이 두 이미지를 사용하여 비교하기 위한 시각화를 나타낼 수 있습니다.

In [ ]:
result = model.signatures['default'](tf.constant(input_img))
generated_image =  result['default']
masked_image = result['masked_image']

visualize_output_comparison(input_img, masked_image, generated_image)